In [2]:
import math
import heapq

# -------------------------------
# 1) Distance Metrics
# -------------------------------
def dist_l2(p, q):
    """Euclidean (L2) distance in 2D."""
    return math.sqrt((p[0] - q[0])**2 + (p[1] - q[1])**2)

def dist_l1(p, q):
    """Manhattan (L1) distance in 2D."""
    return abs(p[0] - q[0]) + abs(p[1] - q[1])

# -------------------------------
# 2) K-d Tree Node
# -------------------------------
class KDNode:
    def __init__(self, point, left=None, right=None):
        self.point = point
        self.left = left
        self.right = right

# -------------------------------
# 3) Build K-d Tree
# -------------------------------
def build_kd_tree(points, depth=0):
    """
    Build a k-d tree from a list of 2D points.
    Splits on x at even depth, y at odd depth.
    """
    if not points:
        return None
    
    # Alternate between x (dim=0) and y (dim=1)
    axis = depth % 2
    
    # Sort points by the axis coordinate
    points.sort(key=lambda p: p[axis])
    
    # Find median index
    median_idx = len(points) // 2
    
    # Create node
    node = KDNode(
        point=points[median_idx],
        left=build_kd_tree(points[:median_idx], depth + 1),
        right=build_kd_tree(points[median_idx+1:], depth + 1)
    )
    
    return node

# -------------------------------
# 4) k-NN Search in K-d Tree
# -------------------------------
def knn_search(root, query_point, k, metric='L2'):
    """
    Return the k nearest neighbors of query_point in the k-d tree.
    
    metric can be 'L2' or 'L1'.
    """
    if metric == 'L2':
        dist_fn = dist_l2
    else:  # 'L1'
        dist_fn = dist_l1

    # We will keep track of (negative_distance, point) in a max-heap (Python heapq is a min-heap, so use negative distance)
    best = []  # will store up to k points
    
    def search_recursive(node, depth=0):
        if node is None:
            return
        
        axis = depth % 2
        pivot = node.point
        
        # 1) Update current best list (if needed)
        d = dist_fn(query_point, pivot)
        
        if len(best) < k:
            # We haven't found k points yet, push this one
            heapq.heappush(best, (-d, pivot))
        else:
            # If this point is closer than the worst in best, replace
            if d < -best[0][0]:  # Compare with negative distance
                heapq.heapreplace(best, (-d, pivot))
        
        # 2) Determine which side to recurse down
        # Compare query vs. pivot on the current axis
        diff = query_point[axis] - pivot[axis]
        
        # Decide which subtree is "primary" and which is "secondary"
        if diff <= 0:
            # query < pivot on this axis => go left first
            search_recursive(node.left, depth + 1)
            # Potentially need to check right side if close enough
            # If the absolute diff is smaller than the furthest best distance,
            # we might need to explore the other side
            if len(best) < k or abs(diff) < -best[0][0]:
                search_recursive(node.right, depth + 1)
        else:
            # query >= pivot => go right first
            search_recursive(node.right, depth + 1)
            # Check left side
            if len(best) < k or abs(diff) < -best[0][0]:
                search_recursive(node.left, depth + 1)
    
    # Initiate recursive search
    search_recursive(root, 0)
    
    # 'best' is a heap of (negative_distance, point), so convert and sort by distance ascending
    results = [(-nd, pt) for (nd, pt) in best]
    results.sort(key=lambda x: x[0])  # sort by distance
    
    return results  # returns list of (distance, point)

# -------------------------------
# 5) Example Usage
# -------------------------------
if __name__ == '__main__':
    # Example data points in 2D
    data_points = [
        (0.1, 0.2), (0.4, 0.4), (0.2, 0.4), (0.3, 0.5),
        (0.35, 0.25), (0.25, 0.45), (0.18, 0.38), (0.45, 0.19),
        (0.33, 0.47), (0.27, 0.42)
    ]
    
    # Build the k-d tree
    kd_tree_root = build_kd_tree(data_points)
    
    # Query point
    A = (0.3, 0.5)
    
    # Find the 3 nearest neighbors using L2 (Euclidean)
    knn_l2 = knn_search(kd_tree_root, A, k=3, metric='L2')
    print("3-NN (L2) results:")
    for dist_val, pt in knn_l2:
        print(f"Distance={dist_val:.4f}, Point={pt}")
    
    # Find the 3 nearest neighbors using L1 (Manhattan)
    knn_l1 = knn_search(kd_tree_root, A, k=3, metric='L1')
    print("\n3-NN (L1) results:")
    for dist_val, pt in knn_l1:
        print(f"Distance={dist_val:.4f}, Point={pt}")


3-NN (L2) results:
Distance=0.0000, Point=(0.3, 0.5)
Distance=0.0424, Point=(0.33, 0.47)
Distance=0.0707, Point=(0.25, 0.45)

3-NN (L1) results:
Distance=0.0000, Point=(0.3, 0.5)
Distance=0.0600, Point=(0.33, 0.47)
Distance=0.1000, Point=(0.25, 0.45)


In [3]:
# Example data points in 2D
data_points = [
    (0.1, 0.2), (0.4, 0.4), (0.2, 0.4), (0.3, 0.5),
    (0.35, 0.25), (0.25, 0.45), (0.18, 0.38), (0.45, 0.19),
    (0.33, 0.47), (0.27, 0.42)
]

# Build the k-d tree
kd_tree_root = build_kd_tree(data_points)

# Query point
A = (0.3, 0.5)

# Find the 3 nearest neighbors using L2 (Euclidean)
knn_l2 = knn_search(kd_tree_root, A, k=3, metric='L2')
print("3-NN (L2) results:")
for dist_val, pt in knn_l2:
    print(f"Distance={dist_val:.4f}, Point={pt}")

# Find the 3 nearest neighbors using L1 (Manhattan)
knn_l1 = knn_search(kd_tree_root, A, k=3, metric='L1')
print("\n3-NN (L1) results:")
for dist_val, pt in knn_l1:
    print(f"Distance={dist_val:.4f}, Point={pt}")


3-NN (L2) results:
Distance=0.0000, Point=(0.3, 0.5)
Distance=0.0424, Point=(0.33, 0.47)
Distance=0.0707, Point=(0.25, 0.45)

3-NN (L1) results:
Distance=0.0000, Point=(0.3, 0.5)
Distance=0.0600, Point=(0.33, 0.47)
Distance=0.1000, Point=(0.25, 0.45)
